## Pré-Processamento

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime

dados = sqlite3.connect('dados.db')
cursor = dados.cursor()
# Consulta teste realizada utilizando somente SQLite
#cursor.execute("SELECT identificação, situacao, destino, transicao FROM log WHERE assunto='REGIME ESPECIAL DE TRIBUTAÇÃO - ALIMENTOS E CONGÊNERES' AND ano='2018' ORDER BY datahoracadastro AND identificação")
#print(cursor.fetchall())
df = pd.read_sql_query("SELECT identificação, situacao, duracao, datahoracadastro FROM log WHERE assunto='REGIME ESPECIAL DE TRIBUTAÇÃO - ALIMENTOS E CONGÊNERES' ORDER BY datahoracadastro AND identificação",dados)
print('Número de eventos: ', len(df))
print('Número de traces: ', len(df['identificação'].unique()))

Número de eventos:  34110
Número de traces:  1391


In [2]:
df = df.rename(columns={'identificação':'caseId','datahoracadastro':'cadastro'}) 
df.shape
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34110 entries, 0 to 34109
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   caseId    34110 non-null  object
 1   situacao  34110 non-null  object
 2   cadastro  34110 non-null  object
dtypes: object(3)
memory usage: 799.6+ KB


caseId      0
situacao    0
cadastro    0
dtype: int64

In [3]:
#Convertendo datahoracadastro em datetime64
from datetime import datetime
df['cadastro'] = pd.to_datetime(df['cadastro'], errors='coerce')
'''
#Convertendo identificação de object para numerico (float64)
df['caseId'] = df['caseId'].str.replace('/','.') 
df['caseId'] = df['caseId'].apply(pd.to_numeric)
i = 0
for i in range(len(df.index)):
    df['caseId'][i] = round(df['caseId'][i], 0)

#Convertendo identificação de float64 para numerico (int64)
df['caseId'] = df['caseId'].astype(np.int64)
'''
df.head(8)

,caseId,situacao,cadastro
0,02060847/2015,PROTOCOLIZADO,2015-04-08 14:40:35
1,02060847/2015,DISTRIBUÍDO,2015-04-08 14:40:36
2,02060847/2015,DISTRIBUÍDO,2015-05-05 17:05:03
3,02060847/2015,DOCUMENTO ASSINADO PELO CONTRIBUINTE,2015-05-08 16:22:45
4,02060847/2015,DOCUMENTO ASSINADO PELO CONTRIBUINTE,2015-05-21 11:19:58
5,02060847/2015,DESPACHO ASSINADO,2015-05-21 16:28:16
6,02060847/2015,AGUARDANDO ASSINATURA DO ORIENTADOR,2015-05-21 16:40:08
7,02060847/2015,ANALISADO,2015-05-22 16:42:53


In [4]:
# REMOVER SEQUÊNCIAS DE ATIVIDADES IGUAIS DE UM MESMO CASE_ID. ATRIBIR ULTIMA DATAHORACADASTRO DA SEQUÊNCIA. 
duplicated = (df['situacao'] == df['situacao'].shift(1)) & (df['caseId'] == df['caseId'].shift(1))
first_id = 0
for id in df.index:
  if duplicated[id]:
    df.at[first_id,'cadastro'] = df.loc[id,'cadastro']
  else:
    first_id = id
df = df[~duplicated]

In [5]:
df= df.to_numpy()
df = pd.DataFrame(df)
df = df.rename(columns={0:'caseId',1:'situacao',2:'cadastro'}) 

In [6]:
df.head()

,caseId,situacao,cadastro
0,02060847/2015,PROTOCOLIZADO,2015-04-08 14:40:35
1,02060847/2015,DISTRIBUÍDO,2015-05-05 17:05:03
2,02060847/2015,DOCUMENTO ASSINADO PELO CONTRIBUINTE,2015-05-21 11:19:58
3,02060847/2015,DESPACHO ASSINADO,2015-05-21 16:28:16
4,02060847/2015,AGUARDANDO ASSINATURA DO ORIENTADOR,2015-05-21 16:40:08


## Preparando Dados de Entrada 

In [7]:
#Filtrando Atividades
atividades = df['situacao'].unique()

In [8]:
#preenchimento das matrizes com 1 caso ocorra atividade no dataframe avaliado
def trasforma(x,identificador):
    groups = df.groupby(df.caseId)
    grupo = groups.get_group(identificador)
    linha = 0
    j=0
    for j in grupo['situacao']:
        k = 0
        while(k<len(atividades)):
            if(j== atividades[k]):
                x[linha][k]=1
                linha+=1
                k=len(atividades)
            else:
                k+= 1      
    return x

def cria_coluna_caseId(linhas, identificador):
    case = []
    for i in range(linhas):
        case.append(identificador)
    return case

In [9]:
#Buscando o tamamjo do maior caso para definir as linhas de cada matriz
i = 0
groups = df.groupby(df.caseId)
total = df['caseId'].unique()
linhas = 0
for i in range(df['caseId'].nunique()):
    grupo = groups.get_group(total[i])
    if(grupo.shape[0]> linhas):
        linhas = grupo.shape[0]

#O total de colunas é o total de atividades existentes
colunas = np.size(atividades)

#Criando a matriz 
matriz = np.zeros(shape=(linhas,colunas))


j = 0
identificadores = df['caseId'].unique()
df_geral = []
prox = 0
for j in range(df['caseId'].nunique()):
    
    x = matriz
    x = trasforma(x,identificadores[j])
    
    #y = pd.DataFrame(cria_coluna_caseId(linhas, identificadores[j]))
    #x = np.append(y, x, axis = 1)
    #x = pd.DataFrame(x)
    #x.columns = atividades
    #x.insert(0,'caseId',y)
    df_geral.append(x)

In [10]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [11]:
dados = df_geral
dataset = []
for k in range(len(df_geral)):
    amostra = []
    for i in dados[k]:
        for j in i:
            amostra.append(j)
    dataset.append(amostra)
dataset = torch.Tensor(dataset)
print(len(dataset[1]))
print(dataset[1])

4130
tensor([1., 0., 0.,  ..., 0., 0., 0.])


In [12]:
data_train = dataset[:2000]
train_loader = DataLoader(data_train, batch_size=20, shuffle=True)

data_test = dataset
test_loader = DataLoader(data_test, batch_size=20, shuffle=True)

## Construindo Autoencoder 

In [13]:
# get_device() retorna o dispositivo GPU se tiver, se não retorna CPU
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

In [14]:
tam = len(dataset[1])
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # encoder
        self.enc1 = nn.Linear(in_features=tam, out_features=64)
        self.enc2 = nn.Linear(in_features=64, out_features=32)
        self.enc3 = nn.Linear(in_features=32, out_features=16)
        # decoder 
        self.dec1 = nn.Linear(in_features=16, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=tam)
        
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        return x
net = Autoencoder().to(get_device())
print(net)

Autoencoder(
  (enc1): Linear(in_features=4130, out_features=64, bias=True)
  (enc2): Linear(in_features=64, out_features=32, bias=True)
  (enc3): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=4130, bias=True)
)


In [15]:
#Definindo, a função de perda e o otimizador para nossa rede
criterio = nn.MSELoss() #Erro quadradico médio
otimizador = optim.Adam(net.parameters(), lr=0.001)

#Função de treinamento do modelo
def train(model, train_loader , criterio, otimizador):
    model.train()
    cumloss = 0.0
    for x in train_loader:
        x = x.to(get_device())
        
        pred = model(x)
        loss = criterio(pred,x)
        
        loss.backward()
        otimizador.step()
    
        cumloss+= loss.item()
    return cumloss/len(train_loader)

#Função de teste do modelo
def test(model, test_loader , criterio, otimizador):
    model.eval()
    cumloss = 0.0
    for x in test_loader:
        x = x.to(get_device())
        
        pred = model(x)
        loss = criterio(pred,x)
        
        cumloss+= loss.item()
    return cumloss/len(train_loader)

## Testando modelo

In [16]:
epocas = 100
for i in range(epocas):
    train_loss = train(net, train_loader, criterio,otimizador)
    if i % 10 == 0:
        print(f"Epoca: {i}; Train Loss: {train_loss}")
test_loss = test(net, test_loader, criterio,otimizador)
print(f"Test Loss: {test_loss}")

Epoca: 0; Train Loss: 0.17640347353049687
Epoca: 10; Train Loss: 0.03877656034060887
Epoca: 20; Train Loss: 0.03776882051357201
Epoca: 30; Train Loss: 0.034959935742829525
Epoca: 40; Train Loss: 0.050170722337705746
Epoca: 50; Train Loss: 0.03986935551677431
Epoca: 60; Train Loss: 0.029165887539940222
Epoca: 70; Train Loss: 0.0323862148714917
Epoca: 80; Train Loss: 0.03159003880407129
Epoca: 90; Train Loss: 0.03443556970783642
Test Loss: 0.034020960330963135
